# Trabalho Final FIA — Raciocínio Espacial Neuro-Simbólico com LTNtorch

# Equipe:
# Kaio Cavalcante de Carvalho 22351230
# Luis Henrique de Carvalho Ribeiro 22352930
# Jorge harrison de Oliveira Pereira Junior 22551159

In [ ]:
import os
os.environ["LTN_BACKEND"] = "torch"


## 1) Imports e utilidades

In [ ]:
import math
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import ltn


In [ ]:
print("tf" in dir(ltn.fuzzy_ops))


False


## 2) Dataset (manual do PDF e gerador aleatório)

In [ ]:
COLUMNS = ["x","y","red","green","blue","circle","square","cylinder","cone","triangle","size"]

def load_manual_dataset_from_pdf_table():
    rows = [
        [0.15,0.85,1,0,0,1,0,0,0,0,0],
        [0.30,0.90,0,1,0,0,1,0,0,0,1],
        [0.45,0.88,0,0,1,0,0,1,0,0,0],
        [0.60,0.90,1,0,0,0,0,0,1,0,1],
        [0.78,0.85,0,1,0,0,0,0,0,1,0],
        [0.20,0.72,0,0,1,1,0,0,0,0,1],
        [0.35,0.75,1,0,0,0,1,0,0,0,0],
        [0.50,0.70,0,1,0,0,0,1,0,0,1],
        [0.65,0.72,0,0,1,0,0,0,1,0,0],
        [0.80,0.75,1,0,0,0,0,0,0,1,1],
        [0.15,0.55,0,1,0,1,0,0,0,0,0],
        [0.30,0.60,0,0,1,0,1,0,0,0,1],
        [0.45,0.58,1,0,0,0,0,1,0,0,0],
        [0.60,0.60,0,1,0,0,0,0,1,0,1],
        [0.78,0.55,0,0,1,0,0,0,0,1,0],
        [0.15,0.40,1,0,0,1,0,0,0,0,1],
        [0.30,0.42,0,1,0,0,1,0,0,0,0],
        [0.45,0.38,0,0,1,0,0,1,0,0,1],
        [0.60,0.42,1,0,0,0,0,0,1,0,0],
        [0.80,0.40,0,1,0,0,0,0,0,1,1],
        [0.20,0.22,0,0,1,1,0,0,0,0,0],
        [0.35,0.25,1,0,0,0,1,0,0,0,1],
        [0.50,0.20,0,1,0,0,0,1,0,0,0],
        [0.65,0.25,0,0,1,0,0,0,1,0,1],
        [0.80,0.22,1,0,0,0,0,0,0,1,0],
    ]
    df = pd.DataFrame(rows, columns=COLUMNS)
    return df

def generate_random_dataset(n=25, seed=0):
    rng = np.random.default_rng(seed)
    xs = rng.uniform(0.05, 0.95, size=n)
    ys = rng.uniform(0.05, 0.95, size=n)

    colors = rng.integers(0, 3, size=n)
    shapes = rng.integers(0, 5, size=n)
    size = rng.integers(0, 2, size=n)

    df = pd.DataFrame(0.0, index=np.arange(n), columns=COLUMNS)
    df["x"] = xs
    df["y"] = ys
    for i in range(n):
        df.loc[i, ["red","green","blue"][colors[i]]] = 1.0
        df.loc[i, ["circle","square","cylinder","cone","triangle"][shapes[i]]] = 1.0
        df.loc[i, "size"] = float(size[i])
    return df

manual_df = load_manual_dataset_from_pdf_table()
manual_df.head()

,x,y,red,green,blue,circle,square,cylinder,cone,triangle,size
0,0.15,0.85,1,0,0,1,0,0,0,0,0
1,0.30,0.90,0,1,0,0,1,0,0,0,1
2,0.45,0.88,0,0,1,0,0,1,0,0,0
3,0.60,0.90,1,0,0,0,0,0,1,0,1
4,0.78,0.85,0,1,0,0,0,0,0,1,0


## 3) Ground truth das relações espaciais

In [ ]:
def gt_left_of(a, b, eps=1e-6):
    return float(a["x"] < b["x"] - eps)

def gt_right_of(a, b, eps=1e-6):
    return float(a["x"] > b["x"] + eps)

def gt_below(a, b, eps=1e-6):
    return float(a["y"] < b["y"] - eps)

def gt_above(a, b, eps=1e-6):
    return float(a["y"] > b["y"] + eps)

def gt_close_to(a, b, thr=0.18):
    dx = float(a["x"] - b["x"])
    dy = float(a["y"] - b["y"])
    d = math.sqrt(dx*dx + dy*dy)
    return float(d < thr)

def gt_in_between(x, y, z):
    return float((gt_left_of(y, x) and gt_right_of(z, x)) or (gt_left_of(z, x) and gt_right_of(y, x)))

def gt_same_size(a, b):
    return float(a["size"] == b["size"])

## 4) Modelos neurais para Predicados (LTNtorch)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class MLP(nn.Module):
    def __init__(self, in_dim, hidden=(32, 32)):
        super().__init__()
        layers=[]
        last=in_dim
        for h in hidden:
            layers += [nn.Linear(last,h), nn.ReLU()]
            last=h
        layers += [nn.Linear(last,1)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.net(x))


class MLP2(nn.Module):
    def __init__(self, in_dim=11, hidden=(32, 32)):
        super().__init__()
        self.net = MLP(in_dim * 2, hidden)

    def forward(self, x, y):
        return self.net(torch.cat([x, y], dim=1))


class MLP3(nn.Module):
    def __init__(self, in_dim=11, hidden=(32, 32)):
        super().__init__()
        self.net = MLP(in_dim * 3, hidden)

    def forward(self, x, y, z):
        return self.net(torch.cat([x, y, z], dim=1))

isCircle   = ltn.Predicate(MLP(11).to(DEVICE))
isSquare   = ltn.Predicate(MLP(11).to(DEVICE))
isCylinder = ltn.Predicate(MLP(11).to(DEVICE))
isCone     = ltn.Predicate(MLP(11).to(DEVICE))
isTriangle = ltn.Predicate(MLP(11).to(DEVICE))
isSmall    = ltn.Predicate(MLP(11).to(DEVICE))
isBig      = ltn.Predicate(MLP(11).to(DEVICE))
leftOf   = ltn.Predicate(MLP2(11).to(DEVICE))
rightOf  = ltn.Predicate(MLP2(11).to(DEVICE))
below    = ltn.Predicate(MLP2(11).to(DEVICE))
above    = ltn.Predicate(MLP2(11).to(DEVICE))
canStack = ltn.Predicate(MLP2(11).to(DEVICE))
inBetween = ltn.Predicate(MLP3(11).to(DEVICE))
class CloseToKernel(nn.Module):
    def forward(self, x, y):
        x1, y1 = x[:, 0], x[:, 1]
        x2, y2 = y[:, 0], y[:, 1]
        d2 = (x1-x2)**2 + (y1-y2)**2
        return torch.exp(-2.0 * d2).unsqueeze(-1)

closeTo = ltn.Predicate(CloseToKernel().to(DEVICE))


class SameSizeKernel(nn.Module):
    def forward(self, x, y):
        s1 = x[:, 10]
        s2 = y[:, 10]
        return (1.0 - torch.abs(s1 - s2)).unsqueeze(-1)

sameSize = ltn.Predicate(SameSizeKernel().to(DEVICE))


## 5) Construção da KB e Axiomas

In [ ]:
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Or  = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Equiv = ltn.Connective(ltn.fuzzy_ops.Equiv(and_op=ltn.fuzzy_ops.AndProd(),
                                         implies_op=ltn.fuzzy_ops.ImpliesReichenbach()))

Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="e")
satAgg = ltn.fuzzy_ops.SatAgg()


def build_axioms(x, y, z):
    axioms = []
    shapes = [isCircle, isSquare, isCylinder, isCone, isTriangle]
    for i in range(len(shapes)):
        for j in range(i + 1, len(shapes)):
            axioms.append(Forall(x, Not(And(shapes[i](x), shapes[j](x)))))
    axioms.append(
        Forall(
            x,
            Or(
                Or(shapes[0](x), shapes[1](x)),
                Or(Or(shapes[2](x), shapes[3](x)), shapes[4](x))
            )
        )
    )
    axioms.append(Forall(x, Not(And(isSmall(x), isBig(x)))))
    axioms.append(Forall(x, Or(isSmall(x), isBig(x))))
    axioms.append(Forall(x, Not(leftOf(x, x))))
    axioms.append(Forall([x, y], Implies(leftOf(x, y), Not(leftOf(y, x)))))
    axioms.append(Forall([x, y], Equiv(leftOf(x, y), rightOf(y, x))))
    axioms.append(Forall([x, y, z], Implies(And(leftOf(x, y), leftOf(y, z)), leftOf(x, z))))
    axioms.append(Forall([x, y], Equiv(below(x, y), above(y, x))))
    axioms.append(Forall([x, y, z], Implies(And(below(x, y), below(y, z)), below(x, z))))
    ss = sameSize(x, y)
    vars_xy = ss.free_vars

    close_xy = (torch.abs(x.value[:, 0].unsqueeze(1) - y.value[:, 0].unsqueeze(0)) < 0.08).float()
    if vars_xy[0] == y.free_vars[0] and vars_xy[1] == x.free_vars[0]:
        close_xy = close_xy.t()

    close_obj = ltn.LTNObject(close_xy, vars_xy)

    stable = Or(ss, close_obj)



    axioms.append(
        Forall(
            [x, y],
            Implies(
                And(
                    And(Not(isCone(y)), Not(isTriangle(y))),
                    And(above(x, y), stable)
                ),
                canStack(x, y)
            )
        )
    )

    return axioms


## 6) Supervisão fraca + treinamento da KB

In [ ]:
def df_to_tensor(df: pd.DataFrame) -> torch.Tensor:
    return torch.tensor(df[COLUMNS].values, dtype=torch.float32, device=DEVICE)

def make_supervision_losses(df_train: pd.DataFrame):
    X = df_to_tensor(df_train)
    obj = ltn.Variable("x", X)

    def bce(pred, target):
        return nn.BCELoss()(pred.value.view(-1, 1), target)



    losses = {}
    losses["circle"]   = bce(isCircle(obj),   X[:,5:6])
    losses["square"]   = bce(isSquare(obj),   X[:,6:7])
    losses["cylinder"] = bce(isCylinder(obj), X[:,7:8])
    losses["cone"]     = bce(isCone(obj),     X[:,8:9])
    losses["triangle"] = bce(isTriangle(obj), X[:,9:10])
    losses["big"]   = bce(isBig(obj),   X[:,10:11])
    losses["small"] = bce(isSmall(obj), 1.0 - X[:,10:11])

    return losses

def make_pair_dataset(df: pd.DataFrame, max_pairs=2000, seed=0):
    rng = np.random.default_rng(seed)
    n = len(df)
    pairs = set()
    while len(pairs) < min(max_pairs, n*(n-1)):
        i, j = int(rng.integers(0,n)), int(rng.integers(0,n))
        if i!=j:
            pairs.add((i,j))
    pairs = list(pairs)

    A = df.iloc[[i for i,j in pairs]].reset_index(drop=True)
    B = df.iloc[[j for i,j in pairs]].reset_index(drop=True)

    y_left  = torch.tensor([[gt_left_of(A.iloc[k], B.iloc[k])] for k in range(len(pairs))], dtype=torch.float32, device=DEVICE)
    y_right = torch.tensor([[gt_right_of(A.iloc[k], B.iloc[k])] for k in range(len(pairs))], dtype=torch.float32, device=DEVICE)
    y_below = torch.tensor([[gt_below(A.iloc[k], B.iloc[k])] for k in range(len(pairs))], dtype=torch.float32, device=DEVICE)
    y_above = torch.tensor([[gt_above(A.iloc[k], B.iloc[k])] for k in range(len(pairs))], dtype=torch.float32, device=DEVICE)

    return A, B, y_left, y_right, y_below, y_above


def train_kb(df: pd.DataFrame, seed=0, epochs=200, lr=1e-3, verbose=False):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    df_train, df_test = train_test_split(df, test_size=0.25, random_state=seed)

    X_train = df_to_tensor(df_train)
    x = ltn.Variable("x", X_train)
    y = ltn.Variable("y", X_train)
    z = ltn.Variable("z", X_train)

    A, B, yL, yR, yB, yA = make_pair_dataset(df_train, seed=seed)
    XA = df_to_tensor(A)
    XB = df_to_tensor(B)

    def bce_tensor(pred, target):
        return nn.BCELoss()(pred.view(-1, 1), target)

    optimizer = torch.optim.Adam(
        [
            *isCircle.model.parameters(),
            *isSquare.model.parameters(),
            *isCylinder.model.parameters(),
            *isCone.model.parameters(),
            *isTriangle.model.parameters(),
            *isSmall.model.parameters(),
            *isBig.model.parameters(),
            *leftOf.model.parameters(),
            *rightOf.model.parameters(),
            *below.model.parameters(),
            *above.model.parameters(),
            *inBetween.model.parameters(),
            *canStack.model.parameters(),
        ],
        lr=lr
    )

    for ep in range(1, epochs + 1):
        optimizer.zero_grad()
        axioms = build_axioms(x, y, z)
        sat = satAgg(*axioms)
        loss_logic = 1.0 - sat
        sup = make_supervision_losses(df_train)
        loss_sup = sum(sup.values())
        predL = leftOf.model(XA, XB)
        predR = rightOf.model(XA, XB)
        predB = below.model(XA, XB)
        predA = above.model(XA, XB)

        loss_pairs = (
            bce_tensor(predL, yL) +
            bce_tensor(predR, yR) +
            bce_tensor(predB, yB) +
            bce_tensor(predA, yA)
        )

        loss = loss_logic + 0.5 * loss_sup + 0.5 * loss_pairs
        loss.backward()
        optimizer.step()

        if verbose and ep % 20 == 0:
            print(f"epoch {ep:03d} | sat={sat.item():.4f} | loss={loss.item():.4f}")

    return df_train, df_test


In [ ]:
run_one(seed=0)


TypeError: Expected parameter 'inputs' to be a tuple of LTNObject, but got [<class 'torch.Tensor'>]

## 7) Métricas + satAgg em teste

In [ ]:
def bin_metrics(y_true, y_pred_prob, thr=0.5):
    y_pred = (y_pred_prob >= thr).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
    return {"accuracy": acc, "precision": prec, "recall": rec, "f1": f1}

def eval_unary(df_test: pd.DataFrame):
    X = df_to_tensor(df_test)
    obj = ltn.Variable("x", X)

    preds = {
        "circle": isCircle(obj).value.detach().cpu().numpy().reshape(-1),
        "square": isSquare(obj).value.detach().cpu().numpy().reshape(-1),
        "cylinder": isCylinder(obj).value.detach().cpu().numpy().reshape(-1),
        "cone": isCone(obj).value.detach().cpu().numpy().reshape(-1),
        "triangle": isTriangle(obj).value.detach().cpu().numpy().reshape(-1),
        "big": isBig(obj).value.detach().cpu().numpy().reshape(-1),
        "small": isSmall(obj).value.detach().cpu().numpy().reshape(-1),
    }
    y = {
        "circle": df_test["circle"].values.astype(int),
        "square": df_test["square"].values.astype(int),
        "cylinder": df_test["cylinder"].values.astype(int),
        "cone": df_test["cone"].values.astype(int),
        "triangle": df_test["triangle"].values.astype(int),
        "big": df_test["size"].values.astype(int),
        "small": (1-df_test["size"].values).astype(int),
    }
    out={}
    for k in preds:
        out[k]=bin_metrics(y[k], preds[k])
    return out

def eval_binary(df_test: pd.DataFrame, seed=0, max_pairs=1000):
    A, B, yL, yR, yB, yA = make_pair_dataset(df_test, max_pairs=max_pairs, seed=seed)
    XA = df_to_tensor(A)
    XB = df_to_tensor(B)

    preds = {
        "leftOf":  leftOf.model(XA, XB).detach().cpu().numpy().reshape(-1),
        "rightOf": rightOf.model(XA, XB).detach().cpu().numpy().reshape(-1),
        "below":   below.model(XA, XB).detach().cpu().numpy().reshape(-1),
        "above":   above.model(XA, XB).detach().cpu().numpy().reshape(-1),
    }

    y = {
        "leftOf":  yL.detach().cpu().numpy().reshape(-1).astype(int),
        "rightOf": yR.detach().cpu().numpy().reshape(-1).astype(int),
        "below":   yB.detach().cpu().numpy().reshape(-1).astype(int),
        "above":   yA.detach().cpu().numpy().reshape(-1).astype(int),
    }

    out = {}
    for k in ["leftOf","rightOf","below","above"]:
        out[k] = bin_metrics(y[k], preds[k])
    return out


def sat_on_test(df_test: pd.DataFrame):
    X = df_to_tensor(df_test)
    x = ltn.Variable("x", X)
    y = ltn.Variable("y", X)
    z = ltn.Variable("z", X)
    axioms = build_axioms(x, y, z)
    return float(satAgg(*axioms).detach().cpu().item())


## 8) Consultas (perguntas) do enunciado

In [ ]:
def query_last_on_left(df):
    X = df_to_tensor(df)
    x = ltn.Variable("x", X)
    y = ltn.Variable("y", X)
    formula = Exists(x, Forall(y, leftOf(x, y)))
    return float(formula.value.detach().cpu().item())


def query_last_on_right(df):
    X = df_to_tensor(df)
    x = ltn.Variable("x", X)
    y = ltn.Variable("y", X)
    formula = Exists(x, Forall(y, rightOf(x, y)))
    return float(formula.value.detach().cpu().item())


def query_small_below_cylinder_left_square(df):
    X = df_to_tensor(df)
    x = ltn.Variable("x", X)
    y = ltn.Variable("y", X)
    z = ltn.Variable("z", X)

    formula = Exists(
        x,
        And(
            isSmall(x),
            And(
                Exists(y, And(isCylinder(y), below(x, y))),
                Exists(z, And(isSquare(z), leftOf(x, z))),
            ),
        ),
    )
    return float(formula.value.detach().cpu().item())


def query_green_cone_in_between(df):
    X = df_to_tensor(df)
    x = ltn.Variable("x", X)
    y = ltn.Variable("y", X)
    z = ltn.Variable("z", X)

    formula = Exists(y, And(isCone(y), Exists(x, Exists(z, inBetween(x, y, z)))))
    return float(formula.value.detach().cpu().item())


## 9) Rodar 5 execuções e gerar tabela final

In [ ]:
def run_one(seed=0, use_manual=False, epochs=200, lr=1e-3):
    df = load_manual_dataset_from_pdf_table() if use_manual else generate_random_dataset(25, seed=seed)
    df_train, df_test = train_kb(df, seed=seed, epochs=epochs, lr=lr, verbose=False)

    out = {
        "seed": seed,
        "sat_test": sat_on_test(df_test),
        "q_last_left": query_last_on_left(df_test),
        "q_last_right": query_last_on_right(df_test),
        "q_composta": query_small_below_cylinder_left_square(df_test),
        "q_cone_verde_between": query_green_cone_in_between(df_test),
    }

    unary = eval_unary(df_test)

    def macro(keys):
        return {
            m: float(np.mean([unary[k][m] for k in keys]))
            for m in ["accuracy","precision","recall","f1"]
        }
    out["shapes_macro"] = macro(["circle","square","cylinder","cone","triangle"])
    out["size_macro"]   = macro(["big","small"])
    return out

results = []
for i in range(5):
    seed = 100 + i
    results.append(run_one(seed=seed, use_manual=False, epochs=250, lr=1e-3))

df_res = pd.DataFrame([{
    "seed": r["seed"],
    "sat_test": r["sat_test"],
    "q_last_left": r["q_last_left"],
    "q_last_right": r["q_last_right"],
    "q_composta": r["q_composta"],
    "q_cone_verde_between": r["q_cone_verde_between"],
    "acc_shapes": r["shapes_macro"]["accuracy"],
    "prec_shapes": r["shapes_macro"]["precision"],
    "rec_shapes": r["shapes_macro"]["recall"],
    "f1_shapes": r["shapes_macro"]["f1"],
    "acc_size": r["size_macro"]["accuracy"],
    "prec_size": r["size_macro"]["precision"],
    "rec_size": r["size_macro"]["recall"],
    "f1_size": r["size_macro"]["f1"],
} for r in results])

df_res

,seed,sat_test,q_last_left,q_last_right,q_composta,q_cone_verde_between,acc_shapes,prec_shapes,rec_shapes,f1_shapes,acc_size,prec_size,rec_size,f1_size
0,100,0.897234,0.281467,0.261023,0.001133,0.172387,1.0,0.8,0.8,0.8,1.0,1.0,1.0,1.0
1,101,0.920024,0.235834,0.270845,0.000059,0.110785,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,102,0.921355,0.211117,0.272985,0.000014,0.000049,1.0,0.6,0.6,0.6,1.0,1.0,1.0,1.0
3,103,0.944306,0.248866,0.294865,0.000029,0.172916,1.0,0.6,0.6,0.6,1.0,1.0,1.0,1.0
4,104,0.891245,0.282442,0.247223,0.003330,0.172237,1.0,0.8,0.8,0.8,1.0,1.0,1.0,1.0


## 10) Executar no dataset manual (do PDF)

In [ ]:
manual_run = run_one(seed=0, use_manual=True, epochs=250, lr=1e-3)
manual_run

{'seed': 0,
 'sat_test': 0.8907451629638672,
 'q_last_left': 0.2844277620315552,
 'q_last_right': 0.24189108610153198,
 'q_composta': 0.0007218718528747559,
 'q_cone_verde_between': 4.798173904418945e-05,
 'shapes_macro': {'accuracy': 1.0, 'precision': 0.8, 'recall': 0.8, 'f1': 0.8},
 'size_macro': {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}